## Imports

In [ ]:
import numpy as np
import rasterio
import gcsfs
from io import BytesIO
import xarray as xr

import utils
from rasterio.windows import Window, get_data_window
from rasterio.transform import from_origin
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
%load_ext autoreload

In [5]:
import importlib
importlib.reload(utils) 

<module 'utils' from '/home/mayuresh/urban-prediction/utils.py'>

### Utils

In [ ]:
def create_windows():
    # Generate windows for each grid
    windows = []
    for i in range(0, height-block_size+1, block_size):
        for j in range(0, width-block_size+1, block_size):
            windows.append(Window(j, i, block_size, block_size))
    return 

def get_train_val_windows():
    return 

def get_sampled_data(data, windows):
    window_data = []
    new_data = np.full(data.shape, np.nan, data.dtype)
    for window in windows:
        row_start = int(window.row_off)
        row_stop = int(window.row_off + window.height)
        col_start = int(window.col_off)
        col_stop = int(window.col_off + window.width)
        new_data[:, row_start:row_stop, col_start:col_stop] = data[:, row_start:row_stop, col_start:col_stop].copy()

    return new_data

def add_padding(test_features, test_label):
    # Padding the arrays to ensure they are of same shape
    if test_features.shape[2]  < test_label.shape[2]:
        padding = test_label.shape[2] - test_features.shape[2]
        test_features_padded = np.pad(test_features, ((0, 0), (0, 0), (0, padding)), mode='constant', constant_values=np.nan)
    return test_features_padded

### GCS Paths

In [5]:
data_dir = "gs://earth-engine-seminar/urbanization/data/export_22122024"
labels = utils.files_in_dir(data_dir, "label.tif")
features = utils.files_in_dir(data_dir, "feat.tif")

In [117]:
labels

['gs://earth-engine-seminar/urbanization/data/export_22122024/2010-01-01/urban_label.tif',
 'gs://earth-engine-seminar/urbanization/data/export_22122024/2011-01-01/urban_label.tif',
 'gs://earth-engine-seminar/urbanization/data/export_22122024/2012-01-01/urban_label.tif',
 'gs://earth-engine-seminar/urbanization/data/export_22122024/2013-01-01/urban_label.tif',
 'gs://earth-engine-seminar/urbanization/data/export_22122024/2014-01-01/urban_label.tif',
 'gs://earth-engine-seminar/urbanization/data/export_22122024/2015-01-01/urban_label.tif',
 'gs://earth-engine-seminar/urbanization/data/export_22122024/2016-01-01/urban_label.tif',
 'gs://earth-engine-seminar/urbanization/data/export_22122024/2017-01-01/urban_label.tif',
 'gs://earth-engine-seminar/urbanization/data/export_22122024/2018-01-01/urban_label.tif']

In [6]:
test_label = utils.load_tif_data(labels[0])
test_features = utils.load_tif_data(features[0])

In [158]:
temp_feat = utils.load_tiff(features[0])
temp_feat.profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 2238, 'height': 2211, 'count': 9, 'crs': CRS.from_wkt('GEOGCS["unknown",DATUM["unknown",SPHEROID["Spheroid",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'), 'transform': Affine(0.013474729261792823, 0.0, 67.81374585363181,
       0.0, -0.013474729261792823, 37.58330317162592), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}

In [7]:
test_label.shape, test_features.shape  

((1, 2211, 2239), (9, 2211, 2238))

In [8]:
np.unique(test_label), np.unique(test_features)

(array([-1,  0,  1,  2,  3], dtype=int8),
 array([-4.1900002e+01, -4.0200001e+01, -4.0099998e+01, ...,
         7.6085492e+04,  9.1639555e+04,            nan],
       shape=(382460,), dtype=float32))

In [ ]:
# Add nan values for no data regions
test_label = np.where(test_label == -1, np.nan, test_label)

In [10]:
np.unique(test_label)

array([ 0.,  1.,  2.,  3., nan])

## Data Preparation

### Add Padding to make shapes same

In [11]:
# Padding the arrays to ensure they are of same shape
if test_features.shape[2]  < test_label.shape[2]:
    padding = test_label.shape[2] - test_features.shape[2]
    test_features_padded = np.pad(test_features, ((0, 0), (0, 0), (0, padding)), mode='constant', constant_values=np.nan)

In [12]:
stacked_data = np.concatenate((test_label, test_features_padded))

In [13]:
stacked_data.shape

(10, 2211, 2239)

### Remove Nan values

In [14]:
valid_mask = ~np.isnan(stacked_data).any(axis=0)

In [36]:
valid_mask.shape

(2211, 2239)

In [15]:
filtered_stacked_data = np.where(valid_mask, stacked_data, np.nan)

In [ ]:
filtered_stacked_data = stacked_data[:, valid_mask]

In [16]:
filtered_stacked_data.shape

(10, 2211, 2239)

### Create windows

In [17]:
block_coverage = 0.3
total_blocks = 100
height, width = filtered_stacked_data.shape[1], filtered_stacked_data.shape[2]
total_pixels = height*width
pixels_covered = total_pixels*block_coverage
block_size = int(np.sqrt(pixels_covered/total_blocks))

# Generate windows for each grid
windows = []
for i in range(0, height-block_size+1, block_size):
    for j in range(0, width-block_size+1, block_size):
        windows.append(Window(j, i, block_size, block_size))

In [18]:
len(windows)

324

In [ ]:
# Randomly split windows into train and validation
train_windows, val_windows = train_test_split(windows, test_size=0.3, random_state=42)

In [ ]:
train_data, temp_val_data = get_sampled_data(filtered_stacked_data, train_windows)
val_data = get_sampled_data(filtered_stacked_data, val_windows)

In [84]:
train_data.shape, temp_val_data.shape

((10, 2211, 2239), (10, 2211, 2239))

In [ ]:

train_data[1:, :, :].shape

(9, 2211, 2239)

### Create a mask for nan data

In [98]:
train_feat = train_data[1:, :, :]
train_label = train_data[0, :, :]
val_feat = val_data[1:, :, :]
val_label = val_data[0, :, :]

nan_feat = np.any(np.isnan(train_feat),axis=0)
nan_label = np.isnan(train_label)
nan_val_feat = np.any(np.isnan(val_feat),axis=0)
nan_val_label = np.isnan(val_label)

In [86]:
nan_feat.shape, nan_label.shape

((2211, 2239), (2211, 2239))

In [99]:
label_mask = ~nan_label
feat_mask = ~nan_feat

val_label_mask = ~nan_val_label
val_feat_mask = ~nan_val_feat

In [89]:
label_mask.shape, feat_mask.shape

((2211, 2239), (2211, 2239))

In [72]:
train_label.shape, train_feat.shape

((2211, 2239), (9, 2211, 2239))

In [100]:
masked_train_labels = train_label[label_mask]
masked_train_feat = train_feat[:, feat_mask]

masked_val_labels = val_label[val_label_mask]
masked_val_feat_data = val_feat[:, val_feat_mask]

In [101]:
masked_train_labels.shape, masked_train_feat.shape, masked_val_labels.shape, masked_val_feat_data.shape

((2272163,), (9, 2272163), (939464,), (9, 939464))

In [102]:
masked_train_feat.transpose().shape

(2272163, 9)

## Train Model

In [105]:
import lightgbm as lgbm

In [106]:
model = lgbm.LGBMClassifier(objective="multiclass", num_class=4)


In [114]:
X = masked_train_feat.transpose()
y = masked_train_labels

X_test = masked_val_feat_data.transpose()
y_test = masked_val_labels

In [115]:
X.shape, y.shape, X_test.shape, y_test.shape

((2272163, 9), (2272163,), (939464, 9), (939464,))

In [ ]:
model = model.fit(X = np.array(X),
                  y=np.array(y),
                  eval_set=[(np.array(X_test), np.array(y_test))],
                  eval_names=["validation"],
                #   eval_metric=["auc", "l1", "l2"],
                  categorical_feature=[7]
                  )

/home/mayuresh/urban-prediction/urban_venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010277 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1989
[LightGBM] [Info] Number of data points in the train set: 2272163, number of used features: 9
[LightGBM] [Info] Start training from score -6.000023
[LightGBM] [Info] Start training from score -7.358304
[LightGBM] [Info] Start training from score -6.258082
[LightGBM] [Info] Start training from score -0.005044


## Predict Future Urbanization

### prepare data for prediction

In [123]:
pred_label = utils.load_tif_data(labels[5])
pred_feat = utils.load_tif_data(features[5])

In [124]:
pred_label.shape, pred_feat.shape

((1, 2211, 2239), (9, 2211, 2238))

In [129]:
pred_feat_padded = add_padding(pred_feat, pred_label)
pred_stacked_data = np.concatenate((pred_label, pred_feat_padded))
pred_stacked_data.shape

(10, 2211, 2239)

In [132]:
valid_mask = ~np.isnan(stacked_data).any(axis=0)
filtered_stacked_data = np.where(valid_mask, stacked_data, np.nan)
# filtered_stacked_data = stacked_data[:, valid_mask]

In [133]:
filtered_stacked_data.shape

(10, 2211, 2239)

In [136]:
pred_feat = filtered_stacked_data[1:, :, :]
pred_label = filtered_stacked_data[0, :, :]

nan_pred_feat = np.any(np.isnan(pred_feat),axis=0)
nan_pred_label = np.isnan(pred_label)

pred_label_mask = ~nan_pred_label
pred_feat_mask = ~nan_pred_feat

masked_pred_labels = pred_label[pred_label_mask]
masked_pred_feat = pred_feat[:, pred_feat_mask]

In [137]:
masked_pred_labels.shape, masked_pred_feat.shape

((3312854,), (9, 3312854))

### predict future

In [138]:
X_pred = masked_pred_feat.transpose()
y_real = masked_pred_labels

In [139]:
y_pred = model.predict(X_pred)

/home/mayuresh/urban-prediction/urban_venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [141]:
y_pred.shape, y_real.shape

((3312854,), (3312854,))

In [146]:
np.save("/home/mayuresh/urban-prediction/predictions_2010.npy", y_pred)

## Confusion matrix

In [ ]:
cm = confusion_matrix(y_real, y_pred, labels=[0,1,2,3])

In [148]:
cm

array([[   4565,     117,     128,    3015],
       [    126,     570,     112,    1342],
       [     84,      63,    1625,    4845],
       [   5499,    1246,    3119, 3286398]])

In [154]:
report = classification_report(y_real, y_pred, labels=[0,1,2,3])

In [156]:
print(report)

              precision    recall  f1-score   support

           0       0.44      0.58      0.50      7825
           1       0.29      0.27      0.27      2150
           2       0.33      0.25      0.28      6617
           3       1.00      1.00      1.00   3296262

    accuracy                           0.99   3312854
   macro avg       0.51      0.52      0.51   3312854
weighted avg       0.99      0.99      0.99   3312854



## Save predictions

In [176]:
shape = pred_label.shape

In [165]:
valid_pred_mask = pred_label_mask.flatten()

In [180]:
profile = temp_feat.profile
pred_array = np.full((shape[0]* shape[1]), np.nan)

In [175]:
profile['height'], profile['width']

(2211, 2238)

In [173]:
len(valid_pred_mask), pred_array.shape

(4950429, (4948218,))

In [181]:
pred_array[valid_pred_mask] = y_pred
pred_array = pred_array.reshape((shape[0], shape[1]))

In [182]:
pred_array.shape

(2211, 2239)

### Create a tiff

In [ ]:
crs = "EPSG:3857"  # Example CRS
output_path = "test_preds_new.tif"
transform = profile['transform']

In [203]:
with rasterio.open(
    output_path,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,  # Single band
    dtype=pred_array.dtype,
    crs=profile['crs'],
    transform=transform,
) as dst:
    dst.write(pred_array, 1)

In [192]:
np.unique(pred_array)

array([ 0.,  1.,  2.,  3., nan])

## Playground

In [30]:
h_idx = np.random.randint(height-block_size+1, size=10)

In [31]:
h_idx

array([2027, 1015, 1286, 2051, 1881, 1965,    9, 1454, 1795, 1125])

In [188]:
height, width

(2211, 2239)

In [200]:
pred_array.dtype

dtype('float64')